In [1]:
import pyedflib as plib
import numpy as np
import pandas as pd
import os
import glob

In [2]:
edf_file_path = 'zhang-wamsley-2019/data/PSG/subject512_REM.edf'


signals, signals_headers, header = plib.highlevel.read_edf(edf_file=edf_file_path)

In [3]:
d_signals = {f'sensor_{i}':signals[i] for i in range(len(signals))}
len(d_signals['sensor_20'])

29220

In [4]:
signals[6][6]

2.7073991266812216

In [5]:
header['annotations']
# 82680.0

[]

## Generamos los excells para entrenar

#### Recorermos la lista y generamos los excels

In [2]:
import pandas as pd

d_sex = {0:"Hombre",
         1 : "Mujer"}

# Define column names
df_info = pd.read_csv('zhang-wamsley-2019/Records.csv', delimiter=',', header=0)    
df_info

,Filename,Case ID,Subject ID,Experience,Treatment group,Duration,EEG sample rate,Number of EEG channels,Last sleep stage,Has EOG,Has EMG,Has ECG,Proportion artifacts,Time of awakening,Subject age,Subject sex,Subject healthy,Has more data,Remarks
0,subject010_Morning.edf,10_Morning,10,2,NaN,73,400,58,5,1,1,0,NaN,7:17:27 AM,23,0,1,1,NaN
1,subject010_NREM.edf,10_NREM,10,2,NaN,73,400,58,2,1,1,0,NaN,2:07:02 AM,23,0,1,1,NaN
2,subject010_REM.edf,10_REM,10,2,NaN,73,400,58,5,1,1,0,NaN,3:50:48 AM,23,0,1,1,NaN
3,subject010_SO1.edf,10_SO1,10,0,NaN,73,400,58,1,1,1,0,NaN,12:16:08 AM,23,0,1,1,NaN
4,subject010_SO10.edf,10_SO10,10,2,NaN,73,400,58,2,1,1,0,NaN,12:54:12 AM,23,0,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,subject535_SO5.edf,535_SO5,535,2,NaN,73,400,58,0,1,1,0,NaN,12:34:25 AM,21,1,1,1,NaN
304,subject535_SO6.edf,535_SO6,535,2,NaN,73,400,58,1,1,1,0,NaN,12:39:22 AM,21,1,1,1,NaN
305,subject535_SO7.edf,535_SO7,535,2,NaN,73,400,58,1,1,1,0,NaN,12:43:30 AM,21,1,1,1,NaN
306,subject535_SO8.edf,535_SO8,535,2,NaN,73,400,58,1,1,1,0,NaN,12:45:20 AM,21,1,1,1,NaN


In [20]:
folder_path = "zhang-wamsley-2019/data/PSG/"

l_edf_files = glob.glob(folder_path+"*.edf")
len(l_edf_files)

308

In [21]:
folder_path = "zhang-wamsley-2019/data/CSV/"

l_edf_files = glob.glob(folder_path+"*.csv")
len(l_edf_files)

298

In [19]:
for i, edf_file in enumerate(l_edf_files[:]):
    # Load the PSG and Hypnogram files
    signals_edf, signals_headers_edf, header_edf = plib.highlevel.read_edf(edf_file=edf_file)

    user_id = df_info[df_info['Filename'] == os.path.basename(edf_file)]['Subject ID'].values[0]
    sex = df_info[df_info['Filename'] == os.path.basename(edf_file)]['Subject sex'].values[0]
    label = df_info[df_info['Filename'] == os.path.basename(edf_file)]['Experience'].values[0]
    
    d_signals = {f'sensor_{i}':signals_edf[i] for i in range(len(signals_edf))}

    data = pd.DataFrame(d_signals)
    data['user_id'] = user_id
    data['sex'] = sex
    data['label'] = label

    # Save the DataFrame to a CSV file
    saving_path = f'zhang-wamsley-2019/data/CSV/{os.path.basename(edf_file)[:-4]}.csv'
    if len(signals_edf) == 63:
        data.to_csv(saving_path, sep=';', index=False)
        print(f'Saving: {saving_path}  -->>  {i}/{len(l_edf_files)}')
    
    else: 
        print("------------------------------")
        print(f"Not saving: {saving_path}")

Saving: zhang-wamsley-2019/data/CSV/subject026_Morning.csv  -->>  0/308
Saving: zhang-wamsley-2019/data/CSV/subject411_NREM.csv  -->>  1/308
Saving: zhang-wamsley-2019/data/CSV/subject266_Morning.csv  -->>  2/308
Saving: zhang-wamsley-2019/data/CSV/subject185_REM.csv  -->>  3/308
------------------------------
Not saving: zhang-wamsley-2019/data/CSV/subject348_SO2.csv
Saving: zhang-wamsley-2019/data/CSV/subject181_SO1.csv  -->>  5/308
Saving: zhang-wamsley-2019/data/CSV/subject088_REM.csv  -->>  6/308
Saving: zhang-wamsley-2019/data/CSV/subject374_SO3.csv  -->>  7/308
Saving: zhang-wamsley-2019/data/CSV/subject337_SO3.csv  -->>  8/308
Saving: zhang-wamsley-2019/data/CSV/subject535_SO8.csv  -->>  9/308
Saving: zhang-wamsley-2019/data/CSV/subject422_SO4.csv  -->>  10/308
Saving: zhang-wamsley-2019/data/CSV/subject535_REM.csv  -->>  11/308
Saving: zhang-wamsley-2019/data/CSV/subject228_SO3.csv  -->>  12/308
Saving: zhang-wamsley-2019/data/CSV/subject183_NREM.csv  -->>  13/308
Saving: zhan

In [8]:
for i, edf_file in enumerate(l_edf_files[:]):
    # print(f'Converting: {edf_file}   -->> {i}/{len(l_edf_files[:])}')
    # Load the PSG and Hypnogram files
    signals_edf, signals_headers_edf, header_edf = plib.highlevel.read_edf(edf_file=edf_file)
    if len(signals_edf) != 63:
        print(f'Converting: {edf_file}   -->> {i}/{len(l_edf_files[:])}')

Converting: zhang-wamsley-2019/data/PSG/subject348_SO2.edf   -->> 4/308
Converting: zhang-wamsley-2019/data/PSG/subject348_REM.edf   -->> 100/308
Converting: zhang-wamsley-2019/data/PSG/subject348_SO3.edf   -->> 116/308
Converting: zhang-wamsley-2019/data/PSG/subject348_NREM.edf   -->> 139/308
Converting: zhang-wamsley-2019/data/PSG/subject348_SO1.edf   -->> 157/308
Converting: zhang-wamsley-2019/data/PSG/subject348_SO5.edf   -->> 215/308
Converting: zhang-wamsley-2019/data/PSG/subject348_Morning.edf   -->> 241/308
Converting: zhang-wamsley-2019/data/PSG/subject348_SO4.edf   -->> 271/308
Converting: zhang-wamsley-2019/data/PSG/subject348_SO6.edf   -->> 287/308
Converting: zhang-wamsley-2019/data/PSG/subject348_SO7.edf   -->> 300/308


In [19]:
pd.read_csv('zhang-wamsley-2019/data/CSV/26-EEG.csv', delimiter=';', header=0)


,sensor_0,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,...,sensor_56,sensor_57,sensor_58,sensor_59,sensor_60,sensor_61,sensor_62,user_id,sex,label
0,-0.144708,-8.035576,0.047964,3.785330,2.002705,6.854253,5.969601,8.753813,5.037882,-1.497392,...,7.480043,13.145251,0.180076,0.000017,0.286305,11.821848,2.909650,26,0,2
1,-0.686907,-7.733036,0.344537,3.553219,1.383462,6.799748,5.316780,8.922866,4.349304,-1.600628,...,6.619921,12.663695,0.593670,0.000017,0.387269,11.661505,3.188887,26,0,2
2,-1.221779,-7.414847,0.683597,3.517140,1.077483,6.869463,4.776252,9.162862,3.850406,-1.552834,...,5.712191,11.795488,0.839967,0.000017,0.307940,11.472148,3.444189,26,0,2
3,-1.741997,-7.039280,0.699085,3.613352,1.011916,6.979740,4.275692,9.381727,3.535261,-1.375039,...,4.706071,10.677716,0.830673,0.000017,-0.009375,11.133137,3.551097,26,0,2
4,-2.240234,-6.590685,0.301701,3.788938,1.109538,7.068468,3.763713,9.531158,3.387073,-1.085406,...,3.603148,9.505461,0.540228,0.000017,-0.573331,10.592553,3.452167,26,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29215,-4.914593,-9.730845,0.818803,-9.049315,-8.643904,-6.234423,-7.191499,-8.290424,-2.540428,-6.117180,...,5.821690,11.002854,11.832736,0.000017,5.602780,2.729646,2.726151,26,0,2
29216,-3.819205,-8.640657,0.442976,-8.420330,-7.878957,-5.382632,-6.279834,-7.339497,-2.231210,-5.414605,...,6.315229,11.231330,11.019489,0.000017,5.024401,3.264122,2.855398,26,0,2
29217,-2.676191,-7.576549,-0.072591,-7.610949,-6.987247,-4.534644,-5.267295,-6.202913,-1.704651,-4.519898,...,6.907157,11.588103,10.406069,0.000017,4.660930,3.916183,3.128252,26,0,2
29218,-1.551495,-6.564604,-0.326189,-6.664466,-6.031427,-3.625814,-4.220497,-4.916897,-1.017060,-3.477028,...,7.610172,12.059114,10.110977,0.000017,4.617660,4.617110,3.541523,26,0,2


## Generamos los estadisticos de Cassete

In [ ]:
import glob
import os
from matplotlib import pyplot as plt
# Define the pattern to match files ending with '-Hypnogram.edf' in 'sleep-cassette' folder
pattern = 'sleep-cassette/*-Hypnogram.edf'

# Use glob.glob to find all files matching the pattern
file_paths = glob.glob(pattern)

# Initialize a list to store the extracted data
data = []

# Iterate over the list of file paths
for file_path in file_paths:
    # Extract user_id from the file basename, assuming the format '...SC4XX...'
    basename = os.path.basename(file_path)
    user_id = basename[3:5]
    user_night = basename[5]
    
    # Read the EDF file
    _, _, header = plib.highlevel.read_edf(edf_file=file_path)

    # Extract the needed information
    sex = header['sex'] if 'sex' in header else 'Unknown'
    patient_age = header['patientname'].split(' ')[-1].strip('yr') if 'patientname' in header else 'Unknown'
    
    # Append the information to the data list
    for sample in header['annotations']: 
        data.append({
            'user_id': int(user_id),
            'user_night': int(user_night),
            'sex': sex,
            'patient_age': int(patient_age),
            'num_seq': int(sample[1]),
            'sleep_stage': sample[2].split(' ')[-1]
        })


In [ ]:
df = pd.DataFrame(data)
df

### Number of unics users

In [ ]:
# Count unique males and females in the dataset
unique_males = df[df['sex'] == 'Male']['user_id'].nunique()
unique_females = df[df['sex'] == 'Female']['user_id'].nunique()

print(f"Number of unic users: {len(df['user_id'].unique())}")
print(f"Unique Males: {unique_males}")
print(f"Unique Females: {unique_females}")

### Distribution Age and Sex

In [ ]:
# Assuming 'df' is your DataFrame

# Step 1: Bin ages (adjust bins according to your data's age distribution)
age_bins = range(df['patient_age'].min(), df['patient_age'].max() + 10, 10)  # Adjust the bin size as needed
df['age_group'] = pd.cut(df['patient_age'], bins=age_bins, right=False)

# Step 2: Count unique user_ids within each (age group, sex) combination
grouped = df.groupby(['age_group', 'sex'])['user_id'].nunique().unstack(fill_value=0)

colors = ['orange' if sex == 'Female' else 'lightblue' for sex in grouped.columns]

# Step 3: Plot
grouped.plot(kind='bar', stacked=True, figsize=(10, 6), width=0.8, color=colors)
plt.title('Density of Unique User IDs by Age Group and Sex')
plt.xlabel('Age Group')
plt.ylabel('Unique User Count')
plt.xticks(rotation=45)
plt.legend(title='Sex')
plt.tight_layout()
plt.show()

In [ ]:
grouped_df = df.groupby(['user_id', 'user_night', 'sleep_stage']).agg({'num_seq':'sum'})

grouped_df

In [ ]:
grouped_df.unstack(level='sleep_stage').plot(kind='bar', stacked=True, figsize=(10, 6))
plt.title('Number of Sequences by User, Night, and Sleep Stage')
plt.xlabel('User ID, User Night')
plt.ylabel('Number of Sequences')
plt.show()

In [ ]:
import seaborn as sns

# Prepare the data
pivot_table = grouped_df.reset_index().pivot_table(index=['user_id', 'user_night'], columns='sleep_stage', values='num_seq', fill_value=0)

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(pivot_table, annot=True, cmap='viridis')
plt.title('Heatmap of Sequences by User, Night, and Sleep Stage')
plt.ylabel('User ID, User Night')
plt.xlabel('Sleep Stage')
plt.show()

In [ ]:
for user_id in df['user_id'].unique():
    user_df = grouped_df.xs(user_id, level='user_id')
    user_df.unstack(level='sleep_stage').plot(kind='line', marker='o', figsize=(10, 6))
    plt.title(f'Sequences Over Nights for User {user_id}')
    plt.ylabel('Number of Sequences')
    plt.xlabel('User Night')
    plt.legend(title='Sleep Stage')
    plt.show()